In [ ]:
# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=1
    )
    acc = accuracy_score(labels, predictions)

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(df['encoded_labels']), 
    y=df['encoded_labels']
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class weights:", class_weights_dict)

# Convert the dictionary of class weights to a tensor
class_weights_tensor = torch.tensor([class_weights_dict[i] for i in range(len(class_weights_dict))], dtype=torch.float)

# Define the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=15)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",            
    evaluation_strategy="epoch",       
    learning_rate=5e-5,                
    per_device_train_batch_size=16,    
    num_train_epochs=3,                
    weight_decay=0.01,                 
    logging_dir="./logs",              
    logging_steps=10                   
)

# Subclass Trainer to add weighted loss function
class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # Accept arbitrary keyword arguments
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Ensure the class weights are on the same device as the model's output logits
        weights = self.class_weights.to(logits.device)

        # Use CrossEntropyLoss with class weights
        loss_fct = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Instantiate the weighted loss trainer with the class weights
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    class_weights=class_weights_tensor,  # Pass tensor of class weights
    compute_metrics=compute_metrics  # Pass compute_metrics here
)

# Start training
trainer.train()
